In [1]:
from keys import load_api_keys
load_api_keys()

In [2]:
import requests
from newspaper import Article

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_url = "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"

session = requests.Session()

try:
  response = session.get(article_url, headers=headers, timeout=10)
  
  if response.status_code == 200:
      article = Article(article_url)
      article.download()
      article.parse()
      
      print(f"Title: {article.title}")
      print(f"Text: {article.text}")
  else:
      print(f"Failed to fetch article at {article_url}")
except Exception as e:
    print(f"Error occurred while fetching article at {article_url}: {e}")

Title: Meta claims its new AI supercomputer will set records
Text: Ryan Daws is a senior editor at TechForge Media with over a decade of experience in crafting compelling narratives and making complex topics accessible. His articles and interviews with industry leaders have earned him recognition as a key influencer by organisations like Onalytica. Under his leadership, publications have been praised by analyst firms such as Forrester for their excellence and performance. Connect with him on X (@gadget_ry), Bluesky (@gadgetry.bsky.social), and/or Mastodon (@gadgetry@techhub.social)

Meta (formerly Facebook) has unveiled an AI supercomputer that it claims will be the world’s fastest.

The supercomputer is called the AI Research SuperCluster (RSC) and is yet to be fully complete. However, Meta’s researchers have already begun using it for training large natural language processing (NLP) and computer vision models.

RSC is set to be fully built in mid-2022. Meta says that it will be the f

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage


# we get the article data from the scraping part
article_title = article.title
article_text = article.text

# prepare template for prompt
template = """
As an advanced AI, you've been tasked to summarize online articles into bulleted points. Here are a few examples of how you've done this in the past:

Example 1:
Original Article: 'The Effects of Climate Change
Summary:
- Climate change is causing a rise in global temperatures.
- This leads to melting ice caps and rising sea levels.
- Resulting in more frequent and severe weather conditions.

Example 2:
Original Article: 'The Evolution of Artificial Intelligence
Summary:
- Artificial Intelligence (AI) has developed significantly over the past decade.
- AI is now used in multiple fields such as healthcare, finance, and transportation.
- The future of AI is promising but requires careful regulation.

Now, here's the article you need to summarize:

==================
Title: {article_title}

{article_text}
==================

Please provide a summarized version of the article in a bulleted list format.
"""

# Format the Prompt
prompt = template.format(article_title=article.title, article_text=article.text)

messages = [HumanMessage(content=prompt)]

In [4]:
from langchain.chat_models import ChatOpenAI

# load the model
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

# generate summary
summary = chat(messages)
print(summary.content)

/tmp/ipykernel_59258/427776806.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)
/tmp/ipykernel_59258/427776806.py:7: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = chat(messages)


Summary:
- Meta has introduced the AI Research SuperCluster (RSC), claiming it will be the fastest AI supercomputer in the world.
- RSC is currently under construction and is expected to be fully operational by mid-2022.
- The supercomputer aims to train models with trillions of parameters, enhancing capabilities in natural language processing (NLP) and computer vision.
- Meta envisions RSC enabling real-time voice translations for collaborative projects and contributing to the development of the metaverse.
- RSC is projected to be 20 times faster than Meta's existing V100-based clusters and significantly improves training times for large-scale NLP workflows.
- A model with tens of billions of parameters can now train in three weeks, down from nine weeks.
- RSC incorporates security and privacy controls, allowing the use of real-world data for training, including identifying harmful content on Meta's platforms.
- Meta claims this is the first instance of addressing performance, reliabi

Parsing output

In [5]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import validator
from pydantic import BaseModel, Field
from typing import List

# create output parser class
class ArticleSummary(BaseModel):
    title: str = Field(description="Title of the article")
    summary: List[str] = Field(description="Bulleted list summary of the article")

    # validating whether the generated summary has at least three lines
    @validator('summary', allow_reuse=True)
    def has_three_or_more_lines(cls, list_of_lines):
        if len(list_of_lines) < 3:
            raise ValueError("Generated summary has less than three bullet points!")
        return list_of_lines

# set up output parser
parser = PydanticOutputParser(pydantic_object=ArticleSummary)

/tmp/ipykernel_59258/2295145831.py:12: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator('summary', allow_reuse=True)


In [6]:
from langchain_core.prompts.prompt import PromptTemplate
# create prompt template
# notice that we are specifying the "partial_variables" parameter
template = """
You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {article_title}

{article_text}
==================

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["article_title", "article_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Format the prompt using the article title and text obtained from scraping
formatted_prompt = prompt.format_prompt(article_title=article_title, article_text=article_text)

In [10]:
from langchain.chat_models import ChatOpenAI

# instantiate model class
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.0)

# Use the model to generate a summary
output = model(formatted_prompt.to_messages())

# Parse the output into the Pydantic model
parsed_output = parser.parse(output.content.split("\"]}")[0] + "\"]}")
print(parsed_output)

for entry in parsed_output.summary:
    print(entry)

title='Meta claims its new AI supercomputer will set records' summary=['Meta has introduced the AI Research SuperCluster (RSC), claiming it will be the fastest AI supercomputer in the world.', 'RSC is currently under construction and is expected to be fully operational by mid-2022.', 'The supercomputer aims to train models with trillions of parameters, enhancing capabilities in natural language processing and computer vision.', 'Meta envisions RSC enabling real-time voice translations and collaboration in the metaverse.', "RSC is projected to be 20 times faster than Meta's existing V100-based clusters and significantly improves training times for large models.", 'The infrastructure is designed with security and privacy in mind, allowing the use of real-world data for training.', 'Meta believes this is the first instance of addressing performance, reliability, security, and privacy at such a scale.']
Meta has introduced the AI Research SuperCluster (RSC), claiming it will be the fastest